In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
import numpy as np
import glob
import cv2
from tqdm.notebook import tqdm
from collections import OrderedDict

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

import imgaug as ia
import imgaug.augmenters as iaa
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

ROOT_DIR = os.path.abspath('/storage/Mask_RCNN')
sys.path.append(ROOT_DIR)
from mrcnn import utils, visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config

%matplotlib inline

In [ ]:
random.seed(42)
np.random.seed(42)

In [ ]:
class MaskDataset(utils.Dataset):
    def load_dataset(self, dataset_dir, dtype, return_coco=True):
        self.dtype = dtype
        if self.dtype == 'train':
            annotation_path = os.path.join(dataset_dir, 'train.json')
            image_dir = os.path.join(dataset_dir, 'train_images')
        elif self.dtype == 'val':
            annotation_path = os.path.join(dataset_dir, 'val.json')
            image_dir = os.path.join(dataset_dir, 'val_images')

        print('Annotation Path ', annotation_path)
        print('Image Dir ', image_dir)
        assert os.path.exists(annotation_path) and os.path.exists(image_dir)

        self.coco = COCO(annotation_path)
        self.image_dir = image_dir

        classIds = self.coco.getCatIds()
        image_ids = list(self.coco.imgs.keys())

        for _class_id in classIds:
            self.add_class('mask-detection', _class_id, self.coco.loadCats(_class_id)[0]['name'])

        for _img_id in image_ids:
            assert(os.path.exists(os.path.join(image_dir, self.coco.imgs[_img_id]['file_name'])))
            self.add_image(
                'mask-detection', image_id=_img_id,
                path=os.path.join(image_dir, self.coco.imgs[_img_id]['file_name']),
                width=self.coco.imgs[_img_id]['width'],
                height=self.coco.imgs[_img_id]['height'],
                annotations=self.coco.loadAnns(self.coco.getAnnIds(imgIds=[_img_id], catIds=classIds, iscrowd=None)))

        if return_coco:
            return self.coco

    def load_mask(self, image_id):
        image_info = self.image_info[image_id]
        assert image_info['source'] == 'mask-detection'

        instance_masks = []
        class_ids = []
        annotations = self.image_info[image_id]['annotations']
        
        for annotation in annotations:
            class_id = self.map_source_class_id(
                'mask-detection.{}'.format(annotation['category_id']))
            if class_id:
                m = self.annToMask(annotation, image_info['height'], image_info['width'])
                if m.max() < 1:
                    continue

                instance_masks.append(m)
                class_ids.append(class_id)
        
        if class_ids:
            mask = np.stack(instance_masks, axis=2)
            class_ids = np.array(class_ids, dtype=np.int32)
            return mask, class_ids
        else:
            return super(MaskDataset, self).load_mask(image_id)


    def image_reference(self, image_id):
        return 'mask-detection::{}'.format(image_id)

    def annToRLE(self, ann, height, width):
        segm = ann['segmentation']
        if isinstance(segm, list):
            rles = maskUtils.frPyObjects(segm, height, width)
            rle = maskUtils.merge(rles)
        elif isinstance(segm['counts'], list):
            rle = maskUtils.frPyObjects(segm, height, width)
        else:
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, height, width):
        rle = self.annToRLE(ann, height, width)
        m = maskUtils.decode(rle)
        return m

In [ ]:
class MaskConfig(Config):
    NAME = 'mask-detection'
    IMAGES_PER_GPU = 2
    GPU_COUNT = 1
    BACKBONE = 'resnet50'
    NUM_CLASSES = 3  # 1 Background + 2 classes(mask/no_mask)
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    MEAN_PIXEL = np.array([0., 0., 0.])

    STEPS_PER_EPOCH = 340
    VALIDATION_STEPS = 60
    MAX_GT_INSTANCES = 35
    LEARNING_RATE = 0.001
    IMAGE_MAX_DIM = 256
    IMAGE_MIN_DIM = 256
    MINI_MASK_SHAPE = (128, 128)
    RPNNMSTHRESHOLD = 0.7
    DETECTIONMINCONFIDENCE = 0.7
    DETECTIONNMSTHRESHOLD = 0.3
    TRAINROISPER_IMAGE = 200
    RPNTRAINANCHORSPERIMAGE = 320

In [ ]:
config = MaskConfig()
config.display()

In [ ]:
DATASET_DIR = 'data/'

In [ ]:
dataset = MaskDataset()
dataset.load_dataset(DATASET_DIR, 'train')
dataset.prepare()

print('[INFO] Image Count: {}'.format(len(dataset.image_ids)))
print('[INFO] Class Count: {}'.format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print('{:3}. {:50}'.format(i, info['name']))

In [ ]:
image_ids = np.random.choice(dataset.image_ids, 4)
for image_id in image_ids:
    print('[INFO] Image ID: {}'.format(image_id))
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names)

In [ ]:
image_id =  np.random.choice(dataset.image_ids, 1)[0]
image = dataset.load_image(image_id)
mask, class_ids = dataset.load_mask(image_id)
bbox = utils.extract_bboxes(mask)


print('[INFO] Image Shape: {} \tClass ID : {}'.format(mask.shape, class_ids))
print('[INFO] Image ID: {} \tDataset Reference: {}'.format(image_id, dataset.image_reference(image_id)))
log('[INFO] Image', image)
log('[INFO] Mask', mask)
log('[INFO] Class IDs', class_ids)
log('[INFO] BBOX', bbox)

visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

In [ ]:
image_id = np.random.choice(dataset.image_ids, 1)[0]
image = dataset.load_image(image_id)
mask, class_ids = dataset.load_mask(image_id)
original_shape = image.shape

image, window, scale, padding, _ = utils.resize_image(
    image, 
    min_dim=config.IMAGE_MIN_DIM, 
    max_dim=config.IMAGE_MAX_DIM,
    mode=config.IMAGE_RESIZE_MODE)
mask = utils.resize_mask(mask, scale, padding)
bbox = utils.extract_bboxes(mask)

print('[INFO] Image ID: {} \tDataset Reference: {}'.format(image_id, dataset.image_reference(image_id)))
print('[INFO] Original Shape: ', original_shape)
log('[INFO] Image', image)
log('[INFO] Mask', mask)
log('[INFO] Class IDs', class_ids)
log('[INFO] BBOX', bbox)

visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

In [ ]:
image_id = np.random.choice(dataset.image_ids, 1)[0]
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
    dataset, config, image_id, use_mini_mask=False)

log('[INFO] Image', image)
log('[INFO] Class IDs', class_ids)
log('[INFO] BBOX', bbox)
log('[INFO] Mask', mask)
log('[INFO] Image Metas', image_meta)

display_images([image]+[mask[:,:,i] for i in range(min(mask.shape[-1], 7))])

In [ ]:
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

In [ ]:
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
    dataset,
    config,
    image_id,
    augmentation=iaa.Affine(rotate=(-35, 35)),
    use_mini_mask=False
  )

log('[INFO] Mask', mask)
display_images([image]+[mask[:,:,i] for i in range(min(mask.shape[-1], 7))])

In [ ]:
mask = utils.expand_mask(bbox, mask, image.shape)
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

In [ ]:
backbone_shapes = modellib.compute_backbone_shapes(config, config.IMAGE_SHAPE)
anchors = utils.generate_pyramid_anchors(
    config.RPN_ANCHOR_SCALES, 
    config.RPN_ANCHOR_RATIOS,
    backbone_shapes,
    config.BACKBONE_STRIDES, 
    config.RPN_ANCHOR_STRIDE
  )

num_levels = len(backbone_shapes)
anchors_per_cell = len(config.RPN_ANCHOR_RATIOS)
print('[INFO] Anchors Count: ', anchors.shape[0])
print('[INFO] Scales: ', config.RPN_ANCHOR_SCALES)
print('[INFO] Ratios: ', config.RPN_ANCHOR_RATIOS)
print('[INFO] Anchors per Cell: ', anchors_per_cell)
print('[INFO] Levels: ', num_levels)

anchors_per_level = []
for l in range(num_levels):
    num_cells = backbone_shapes[l][0] * backbone_shapes[l][1]
    anchors_per_level.append(anchors_per_cell * num_cells // config.RPN_ANCHOR_STRIDE**2)
    print('Anchors at Level {}: {}'.format(l, anchors_per_level[l]))

In [ ]:
image_id = np.random.choice(dataset.image_ids, 1)[0]
image, image_meta, _, _, _ = modellib.load_image_gt(dataset, config, image_id)
fig, ax = plt.subplots(1, figsize=(10, 10))
ax.imshow(image)
levels = len(backbone_shapes)

for level in range(levels):
    colors = visualize.random_colors(levels)
    level_start = sum(anchors_per_level[:level])
    level_anchors = anchors[level_start:level_start+anchors_per_level[level]]
    print('Level {}. Anchors: {:6}  Feature map Shape: {}'.format(level, level_anchors.shape[0], 
                                                                  backbone_shapes[level]))
    center_cell = backbone_shapes[level] // 2
    center_cell_index = (center_cell[0] * backbone_shapes[level][1] + center_cell[1])
    level_center = center_cell_index * anchors_per_cell 
    center_anchor = anchors_per_cell * (
        (center_cell[0] * backbone_shapes[level][1] / config.RPN_ANCHOR_STRIDE**2) \
        + center_cell[1] / config.RPN_ANCHOR_STRIDE)
    level_center = int(center_anchor)

    for i, rect in enumerate(level_anchors[level_center:level_center+anchors_per_cell]):
        y1, x1, y2, x2 = rect
        p = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, facecolor='none',
                              edgecolor=(i+1)*np.array(colors[level]) / anchors_per_cell)
        ax.add_patch(p)

In [ ]:
random_rois = 2000
g = modellib.data_generator(
    dataset, config, shuffle=True, random_rois=random_rois, 
    batch_size=4,
    detection_targets=True)

In [ ]:
if random_rois:
    [normalized_images, image_meta, rpn_match, rpn_bbox, gt_class_ids, gt_boxes, gt_masks, rpn_rois, rois], \
    [mrcnn_class_ids, mrcnn_bbox, mrcnn_mask] = next(g)
    
    log('[INFO] rois', rois)
    log('[INFO] mrcnn_class_ids', mrcnn_class_ids)
    log('[INFO] mrcnn_bbox', mrcnn_bbox)
    log('[INFO] mrcnn_mask', mrcnn_mask)
else:
    [normalized_images, image_meta, rpn_match, rpn_bbox, gt_boxes, gt_masks], _ = next(g)
    
log('[INFO] gt_class_ids', gt_class_ids)
log('[INFO] gt_boxes', gt_boxes)
log('[INFO] gt_masks', gt_masks)
log('[INFO] rpn_match', rpn_match, )
log('[INFO] rpn_bbox', rpn_bbox)
image_id = modellib.parse_image_meta(image_meta)['image_id'][0]
print('[INFO] Image ID: {} \tDataset Reference: {}'.format(image_id, dataset.image_reference(image_id)))

mrcnn_class_ids = mrcnn_class_ids[:,:,0]

In [ ]:
b = 0
sample_image = modellib.unmold_image(normalized_images[b], config)

indices = np.where(rpn_match[b] == 1)[0]
refined_anchors = utils.apply_box_deltas(anchors[indices], rpn_bbox[b, :len(indices)] * config.RPN_BBOX_STD_DEV)
log('anchors', anchors)
log('refined_anchors', refined_anchors)

positive_anchor_ids = np.where(rpn_match[b] == 1)[0]
print('Positive anchors: {}'.format(len(positive_anchor_ids)))
negative_anchor_ids = np.where(rpn_match[b] == -1)[0]
print('Negative anchors: {}'.format(len(negative_anchor_ids)))
neutral_anchor_ids = np.where(rpn_match[b] == 0)[0]
print('Neutral anchors: {}'.format(len(neutral_anchor_ids)))

for c, n in zip(dataset.class_names, np.bincount(mrcnn_class_ids[b].flatten())):
    if n:
        print('{:23}: {}'.format(c[:20], n))

visualize.draw_boxes(sample_image, boxes=anchors[positive_anchor_ids], 
                     refined_boxes=refined_anchors)

In [ ]:
visualize.draw_boxes(sample_image, boxes=anchors[negative_anchor_ids])

In [ ]:
visualize.draw_boxes(sample_image, boxes=anchors[np.random.choice(neutral_anchor_ids, 100)])

In [ ]:
if random_rois:
    bbox_specific = mrcnn_bbox[b, np.arange(mrcnn_bbox.shape[1]), mrcnn_class_ids[b], :]
    refined_rois = utils.apply_box_deltas(rois[b].astype(np.float32), bbox_specific[:,:4] * config.BBOX_STD_DEV)
    mask_specific = mrcnn_mask[b, np.arange(mrcnn_mask.shape[1]), :, :, mrcnn_class_ids[b]]
    visualize.draw_rois(sample_image, rois[b], refined_rois, mask_specific, mrcnn_class_ids[b], dataset.class_names)
    
    rows = np.ascontiguousarray(rois[b]).view(np.dtype((np.void, rois.dtype.itemsize * rois.shape[-1])))
    _, idx = np.unique(rows, return_index=True)
    print('Unique ROIs: {} out of {}'.format(len(idx), rois.shape[1]))

In [ ]:
if random_rois:
    ids = random.sample(range(rois.shape[1]), 8)

    images = []
    titles = []
    for i in ids:
        image = visualize.draw_box(sample_image.copy(), rois[b,i,:4].astype(np.int32), [255, 0, 0])
        image = visualize.draw_box(image, refined_rois[i].astype(np.int64), [0, 255, 0])
        images.append(image)
        titles.append('ROI {}'.format(i))
        images.append(mask_specific[i] * 255)
        titles.append(dataset.class_names[mrcnn_class_ids[b,i]][:20])

    display_images(images, titles, cols=4, cmap='Blues', interpolation='none')

In [ ]:
if random_rois:
    limit = 10
    temp_g = modellib.data_generator(
        dataset, config, shuffle=True, random_rois=10000, 
        batch_size=1, detection_targets=True)
    total = 0
    for i in range(limit):
        _, [ids, _, _] = next(temp_g)
        positive_rois = np.sum(ids[0] > 0)
        total += positive_rois
        print('{:5} {:5.2f}'.format(positive_rois, positive_rois/ids.shape[1]))
    print('Average percent: {:.2f}'.format(total/(limit*ids.shape[1])))

In [ ]:
dataset_train = MaskDataset()
dataset_train.load_dataset(DATASET_DIR,'train')
dataset_train.prepare()

In [ ]:
dataset_val = MaskDataset()
val_coco = dataset_val.load_dataset(DATASET_DIR,'val')
dataset_val.prepare()

In [ ]:
print('Loading Mask R-CNN model...')
model = modellib.MaskRCNN(mode='training', config=config, model_dir='./')

In [ ]:
try:
    model_path = model.find_last()
    model.load_weights(model_path, by_name=True)
except:
    model.load_weights('mask_rcnn_coco.h5', 
                       by_name=True, 
                       exclude=['mrcnn_class_logits', 'mrcnn_bbox_fc',
                                'mrcnn_bbox', 'mrcnn_mask'])

In [ ]:
augmentation = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flip
    iaa.Flipud(0.5), # vertical flip
#     iaa.CropAndPad(
#         percent=(-0.05, 0.1),
#         pad_mode=ia.ALL,
#         pad_cval=(0, 255)
#     ),
#     iaa.Affine(
#         scale={'x': (0.8, 1.2), 'y': (0.8, 1.2)}, # scale from 80-120%
#         translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to 20 %
#         rotate=(-45, 45), # rotate by -45 to 45
#         shear=(-8, 8),
#         order=[0, 1],
#         cval=(0, 255),
#         mode=ia.ALL
#     )
], random_order=True)

In [ ]:
print('Training network')
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=20,
            layers='4+',
            augmentation=augmentation
           )

In [ ]:
with open(DATASET_DIR + '/val.json') as json_file:
    data = json.load(json_file)

In [ ]:
d = {}
for x in data['categories']:
    d[x['name']] = x['id']

In [ ]:
d

In [ ]:
id_category = [0]
for x in dataset.class_names[1:]:
    id_category.append(d[x])
id_category

In [ ]:
val_images_map_id = {}
for x in data['images']:
    val_images_map_id[x['file_name']] = x['id']

In [ ]:
class InferenceConfig(MaskConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 3  # 1 Background + 61 classes
    IMAGE_MAX_DIM = 256
    IMAGE_MIN_DIM = 256
    NAME = 'mask-detection'
    DETECTIONMINCONFIDENCE = 0.8
    RPNNMSTHRESHOLD = 0.6
    DETECTIONNMSTHRESHOLD = 0.1

In [ ]:
inference_config = InferenceConfig()
inference_config.display()

In [ ]:
model_val = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir='./')
model_path = model_val.find_last()
assert model_path != '', 'Provide path to trained weights'
print('Loading weights from ', model_path)
model_val.load_weights(model_path, by_name=True)

In [ ]:
dataset = dataset_val
fig = plt.figure(figsize=(15, 30))

for i in range(4):

    image_id = random.choice(dataset.image_ids)
    
    original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config, 
                               image_id, use_mini_mask=False)
    
    print(original_image.shape)
    plt.subplot(6, 2, 2*i + 1)
    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                                dataset.class_names, ax=fig.axes[-1])
    
    plt.subplot(6, 2, 2*i + 2)
    results = model_val.detect([original_image])
    r = results[0]
    visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], ax=fig.axes[-1])

In [ ]:
files = os.listdir(DATASET_DIR + 'val_images')
_final_object = []

for f in tqdm(files, total=len(files)):
    try:
        images = [cv2.imread(DATASET_DIR + 'val_images/' + f) ]
        predictions = model_val.detect(images, verbose=0)

        for _idx, r in enumerate(predictions):

            image_id = f.split('.')[0]
            for _idx, class_id in enumerate(r['class_ids']):
                if class_id > 0:
                    mask = r['masks'].astype(np.uint8)[:, :, _idx]
                    bbox = np.around(r['rois'][_idx], 1)
                    bbox = [float(x) for x in bbox]
                    _result = {}
                    _result['image_id'] = val_images_map_id[f]
                    _result['category_id'] = id_category[class_id]
                    _result['score'] = float(r['scores'][_idx])
                    _mask = maskUtils.encode(np.asfortranarray(mask))
                    _mask['counts'] = _mask['counts'].decode('UTF-8')
                    _result['segmentation'] = _mask
                    _result['bbox'] = [bbox[1], bbox[0], bbox[3] - bbox[1], bbox[2] - bbox[0]]
                    _final_object.append(_result)
    except:
        print('Error with',f)
        continue

In [ ]:
fp = open('data/output.json', 'w')
print('Writing JSON...')
fp.write(json.dumps(_final_object))
fp.close()

In [ ]:
submission_file = json.loads(open('data/output.json').read())
len(submission_file)

In [ ]:
GROUND_TRUTH_ANNOTATION_PATH = DATASET_DIR + 'val.json'
ground_truth_annotations = COCO(GROUND_TRUTH_ANNOTATION_PATH)
submission_file = json.loads(open('data/output.json').read())
results = ground_truth_annotations.loadRes(submission_file)
cocoEval = COCOeval(ground_truth_annotations, results, 'bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

In [ ]:
with open(DATASET_DIR + '/test.json') as json_file:
    data = json.load(json_file)

In [ ]:
test_images_map_id={}
for x in data['images']:
    test_images_map_id[x['file_name']] = x['id']

In [ ]:
files = os.listdir(DATASET_DIR + 'test_images')
_final_object = []
for f in tqdm(files,total=len(files)):
    try:
        images = [cv2.imread(DATASET_DIR + 'test_images/' + f)]
        predictions = model_val.detect(images, verbose=0)

        for _idx, r in enumerate(predictions):
            image_id = f.split('.')[0]
            for _idx, class_id in enumerate(r['class_ids']):
                if class_id > 0:
                    mask = r['masks'].astype(np.uint8)[:, :, _idx]
                    bbox = np.around(r['rois'][_idx], 1)
                    bbox = [float(x) for x in bbox]
                    _result = {}
                    _result['image_id'] = test_images_map_id[f]
                    _result['category_id'] = id_category[class_id]
                    _result['score'] = float(r['scores'][_idx])
                    _mask = maskUtils.encode(np.asfortranarray(mask))
                    _mask['counts'] = _mask['counts'].decode('UTF-8')
                    _result['segmentation'] = _mask
                    _result['bbox'] = [bbox[1], bbox[0], bbox[3] - bbox[1], bbox[2] - bbox[0]]
                    _final_object.append(_result)


    except:
        print('Error with',f)
        continue

In [ ]:
fp = open('data/submission.json', 'w')
print('Writing JSON...')
fp.write(json.dumps(_final_object))
fp.close()